In [ ]:
%cd /workspace/examples/cfd/flow_reconstruction_diffusion

## Set up and train

In [ ]:
!./setup_and_train.sh

## Super-resolution (irregular input)

In [ ]:
!python generate_dfsr.py --config conf/config_dfsr_cond_generate.yaml

In [ ]:
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio

base_dir = "outputs/guided_recons_u3232_u3232_t320_r40_w0.0_s0"

batch_dirs = sorted(
    glob.glob(os.path.join(base_dir, "sample_batch*")),
    key=lambda p: int(os.path.basename(p).replace("sample_batch", ""))
)

frames = []
for bd in batch_dirs:
    inp   = np.load(os.path.join(bd, "input_arr.npy"))[0]
    ref   = np.load(os.path.join(bd, "reference_arr.npy"))[0]
    recon = np.load(os.path.join(bd, "sample_arr_run_0_it0.npy"))[0]

    fig, axs = plt.subplots(1, 3, figsize=(9, 3))
    labels = ["Input", "Reconstruction", "Ground truth"]

    for ax, field, label in zip(axs, [inp, recon, ref], labels):
        ax.imshow(field, cmap="twilight", vmin=-23, vmax=23)
        ax.text(
            0.02, 0.98, label,
            transform=ax.transAxes,
            ha="left", va="top",
            fontsize=12, fontweight="bold",
            color="black",
            bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.2"),
        )
        ax.axis("off")

    fig.tight_layout(pad=0.1)
    fig.canvas.draw()
    frames.append(np.asarray(fig.canvas.buffer_rgba())[:, :, :3])
    plt.close(fig)

out_gif = os.path.join(base_dir, "input_recon_gt.gif")
imageio.mimsave(out_gif, frames, duration=0.1)
out_gif

In [ ]:
import base64
from IPython.display import HTML

gif_path = "outputs/guided_recons_u3232_u3232_t320_r40_w0.0_s0/input_recon_gt.gif"
data = base64.b64encode(open(gif_path, "rb").read()).decode("ascii")

HTML(f'<img src="data:image/gif;base64,{data}" />')
